<a href="https://colab.research.google.com/github/tomassalcedas/dataeng/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Aggregations
- Group By
- Windows Functions

# Setting up PySpark

In [2]:
%pip install pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()

# Aggregations

https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions

https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-aggregate.html

In [19]:
sql_query = """CREATE OR REPLACE TEMPORARY VIEW basic_pays AS SELECT * FROM VALUES
('Diane Murphy','Accounting',8435),
('Mary Patterson','Accounting',9998),
('Jeff Firrelli','Accounting',8992),
('William Patterson','Accounting',8870),
('Gerard Bondur','Accounting',11472),
('Anthony Bow','Accounting',6627),
('Leslie Jennings','IT',8113),
('Leslie Thompson','IT',5186),
('Julie Firrelli','Sales',9181),
('Steve Patterson','Sales',9441),
('Foon Yue Tseng','Sales',6660),
('George Vanauf','Sales',10563),
('Loui Bondur','SCM',10449),
('Gerard Hernandez','SCM',6949),
('Pamela Castillo','SCM',11303),
('Larry Bott','SCM',11798),
('Barry Jones','SCM',10586)
AS basic_pays(employee_name, department, salary)"""

# creating temp view
spark.sql(sql_query)

df = spark.table("basic_pays")
df.show()


+-----------------+----------+------+
|    employee_name|department|salary|
+-----------------+----------+------+
|     Diane Murphy|Accounting|  8435|
|   Mary Patterson|Accounting|  9998|
|    Jeff Firrelli|Accounting|  8992|
|William Patterson|Accounting|  8870|
|    Gerard Bondur|Accounting| 11472|
|      Anthony Bow|Accounting|  6627|
|  Leslie Jennings|        IT|  8113|
|  Leslie Thompson|        IT|  5186|
|   Julie Firrelli|     Sales|  9181|
|  Steve Patterson|     Sales|  9441|
|   Foon Yue Tseng|     Sales|  6660|
|    George Vanauf|     Sales| 10563|
|      Loui Bondur|       SCM| 10449|
| Gerard Hernandez|       SCM|  6949|
|  Pamela Castillo|       SCM| 11303|
|       Larry Bott|       SCM| 11798|
|      Barry Jones|       SCM| 10586|
+-----------------+----------+------+



In [20]:
perc_query = """SELECT
    department,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) AS pc1,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pc2,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pc3,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pc4,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) AS pd1,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pd2,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pd3,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pd4
FROM basic_pays
GROUP BY department
ORDER BY department;"""

spark.sql(perc_query).show()

+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|department|    pc1|     pc2|    pc3|     pc4|    pd1|    pd2|    pd3|    pd4|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|Accounting|8543.75| 7838.25| 9746.5|10260.75| 8435.0| 6627.0| 9998.0|11472.0|
|        IT|5917.75|    NULL|7381.25|    NULL| 5186.0|   NULL| 8113.0|   NULL|
|       SCM|10449.0|10786.25|11303.0|11460.75|10449.0|10449.0|11303.0|11798.0|
|     Sales|8550.75|    NULL| 9721.5|    NULL| 6660.0|   NULL|10563.0|   NULL|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+



In [21]:
from pyspark.sql.functions import *
(df
 .groupBy("department")
 .agg(sum("salary").alias("sum_salary"),
      round(avg("salary"),2).alias("avg_salary"),
      min("salary").alias("min_salary"),
      array_agg("employee_name").alias("employees"),
      count(lit("")).alias("count_employees"))
 .filter(col("count_employees") > 2)
 .show(10, False))

+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|department|sum_salary|avg_salary|min_salary|employees                                                                                   |count_employees|
+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|Sales     |35845     |8961.25   |6660      |[Julie Firrelli, Steve Patterson, Foon Yue Tseng, George Vanauf]                            |4              |
|Accounting|54394     |9065.67   |6627      |[Diane Murphy, Mary Patterson, Jeff Firrelli, William Patterson, Gerard Bondur, Anthony Bow]|6              |
|SCM       |51085     |10217.0   |6949      |[Loui Bondur, Gerard Hernandez, Pamela Castillo, Larry Bott, Barry Jones]                   |5              |
+----------+----------+----------+----------+-------------------------

# Question

In [ ]:
# Q1
# Aggregate data by surname
# Calculate highest salary by surname
# Include the respective employee that has the highest salary
# Include department information about this employee
# Count how many employees has that surname
# Put in an array all the first_names of the respective surname ordered


# schema expected:
# surname | count_employees | highest_salary | employee_with_highest_salary | department_with_highest_salary | array_with_all_the_first_names |

In [25]:
from pyspark.sql.functions import *

df = (df.withColumn("surname", split(col("employee_name"), " ").getItem(1))
      .withColumn("first_name", split(col("employee_name"), " ").getItem(0)))

df.show()

+-----------------+----------+------+---------+---+----------+
|    employee_name|department|salary|  surname| rn|first_name|
+-----------------+----------+------+---------+---+----------+
|    Gerard Bondur|Accounting| 11472|   Bondur|  1|    Gerard|
|      Loui Bondur|       SCM| 10449|   Bondur|  2|      Loui|
|       Larry Bott|       SCM| 11798|     Bott|  1|     Larry|
|      Anthony Bow|Accounting|  6627|      Bow|  1|   Anthony|
|  Pamela Castillo|       SCM| 11303| Castillo|  1|    Pamela|
|   Julie Firrelli|     Sales|  9181| Firrelli|  1|     Julie|
|    Jeff Firrelli|Accounting|  8992| Firrelli|  2|      Jeff|
| Gerard Hernandez|       SCM|  6949|Hernandez|  1|    Gerard|
|  Leslie Jennings|        IT|  8113| Jennings|  1|    Leslie|
|      Barry Jones|       SCM| 10586|    Jones|  1|     Barry|
|     Diane Murphy|Accounting|  8435|   Murphy|  1|     Diane|
|   Mary Patterson|Accounting|  9998|Patterson|  1|      Mary|
|  Steve Patterson|     Sales|  9441|Patterson|  2|    

In [26]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, asc

w = Window.partitionBy("surname").orderBy(desc("salary"), asc("employee_name"))

df = df.withColumn("rn", row_number().over(w))
df.show()

+-----------------+----------+------+---------+---+----------+
|    employee_name|department|salary|  surname| rn|first_name|
+-----------------+----------+------+---------+---+----------+
|    Gerard Bondur|Accounting| 11472|   Bondur|  1|    Gerard|
|      Loui Bondur|       SCM| 10449|   Bondur|  2|      Loui|
|       Larry Bott|       SCM| 11798|     Bott|  1|     Larry|
|      Anthony Bow|Accounting|  6627|      Bow|  1|   Anthony|
|  Pamela Castillo|       SCM| 11303| Castillo|  1|    Pamela|
|   Julie Firrelli|     Sales|  9181| Firrelli|  1|     Julie|
|    Jeff Firrelli|Accounting|  8992| Firrelli|  2|      Jeff|
| Gerard Hernandez|       SCM|  6949|Hernandez|  1|    Gerard|
|  Leslie Jennings|        IT|  8113| Jennings|  1|    Leslie|
|      Barry Jones|       SCM| 10586|    Jones|  1|     Barry|
|     Diane Murphy|Accounting|  8435|   Murphy|  1|     Diane|
|   Mary Patterson|Accounting|  9998|Patterson|  1|      Mary|
|  Steve Patterson|     Sales|  9441|Patterson|  2|    

In [27]:
from pyspark.sql.functions import count, max, collect_list, array_sort

agg = (
    df.groupBy("surname")
      .agg(
          count("*").alias("count_employees"),
          max("salary").alias("highest_salary"),
          array_sort(collect_list("first_name")).alias("array_with_all_the_first_names")
      )
)
agg.show()

+---------+---------------+--------------+------------------------------+
|  surname|count_employees|highest_salary|array_with_all_the_first_names|
+---------+---------------+--------------+------------------------------+
|      Bow|              1|          6627|                     [Anthony]|
|    Jones|              1|         10586|                       [Barry]|
|   Bondur|              2|         11472|                [Gerard, Loui]|
|   Murphy|              1|          8435|                       [Diane]|
| Castillo|              1|         11303|                      [Pamela]|
| Firrelli|              2|          9181|                 [Jeff, Julie]|
|   Vanauf|              1|         10563|                      [George]|
|      Yue|              1|          6660|                        [Foon]|
|Patterson|              3|          9998|          [Mary, Steve, Wil...|
| Thompson|              1|          5186|                      [Leslie]|
| Jennings|              1|          8

In [28]:
top_emp = (
    df.filter(col("rn") == 1)
      .select(
          "surname",
          col("employee_name").alias("employee_with_highest_salary"),
          col("department").alias("department_with_highest_salary")
      )
)

In [29]:
result = (
    agg.join(top_emp, "surname", "left")
       .select(
           "surname",
           "count_employees",
           "highest_salary",
           "employee_with_highest_salary",
           "department_with_highest_salary",
           "array_with_all_the_first_names"
       )
)

result.show(truncate=False)

+---------+---------------+--------------+----------------------------+------------------------------+------------------------------+
|surname  |count_employees|highest_salary|employee_with_highest_salary|department_with_highest_salary|array_with_all_the_first_names|
+---------+---------------+--------------+----------------------------+------------------------------+------------------------------+
|Bow      |1              |6627          |Anthony Bow                 |Accounting                    |[Anthony]                     |
|Jones    |1              |10586         |Barry Jones                 |SCM                           |[Barry]                       |
|Bondur   |2              |11472         |Gerard Bondur               |Accounting                    |[Gerard, Loui]                |
|Murphy   |1              |8435          |Diane Murphy                |Accounting                    |[Diane]                       |
|Castillo |1              |11303         |Pamela Castillo     